# Day 2, session 2: Extracting signal tracks from contact matrices

In this session we will extract biologically relevant signal tracks from contact matrices.
We will be working with published data from [Bonev et al. 2017](https://doi.org/10.1016/j.cell.2017.09.043).
We have 2 cool files at 5kb resolution:

* mouse Embryonic stem cells (mESC)
* Neuroprogenitor cells (NPC)

Here, we will be comparing these two stages of differentiation using different features extracted from contact maps.

## Multiresolution files

The cool files provided are at 5kb resolution. There is a trade-off when choosing a resolution for Hi-C contact maps. The higher resolution, the more details can be seen, however this also reduces the contrast, especially at longer ranges. This is visible by eye: At high resolutions, contacts quickly drop to near-zero values when going away from the diagonal. 

Depending on the coverage and signal we are interested in, we might need to use different resolutions. For example, short range signals like TADs or chromatin loops are analysed easily at high resolutions (e.g. 5-10kb), whereas compartments often require using lower resolutions (e.g. 50-100kb).

Fortunately, the cooler API offers a command to quickly generate cool files at multiple-resolutions. These files are called "mcool" files.

In [ ]:
%%bash
cooler zoomify -r 5000,10000,20000,50000,100000 --balance input.cool -o output.mcool

The resulting mcool file will actually contain multiple cool files withing its tree hierarchy. you can inspect it using `cooler tree output.mcool`. You should see a root note named `/resolutions` with children whose name is the resolution (e.g. `5000`). This path within the tree is named the URI any cool within a mcool can be accessed using its URI. The file path and URI are separated using `::/`. For example, to show a region of the matrix at 50kb resolution:

In [ ]:
%%bash
cooler show output.mcool::/resolutions/50000 'chr1:3000000-6000000'

Similarly, to access it from the python API:

In [ ]:
import cooler
clr = cooler.Cooler('output.mcool::/resolutions/50000')

## Insulation score

Insulation scores is used by many methods to measure a "border score". Insulation measures the depletion of contacts across a position, which is typically the case at TAD borders.

To compute insulation, we slide a "diamond" along the diagonal, with the diamond corner touching the diagonal. Here is a basic implementation to understand the concept:

In [ ]:
def insulation(mat, win_size=10):
    score = np.nan * np.ones(mat.shape[0])
    for i in range(0, mat.shape[0]):
        lo = max(0, i + 1 - win_size)
        hi = min(i + win_size, mat.shape[0])
        score[i] = np.nanmean(mat[lo : i + 1, i:hi])
    return score

Fortunately, this is already implemented in some packages such as cooltools:

In [ ]:
cooltools diamond-insulation output.mcool::/resolutions/10000 100000

> What is the consequence of changing the window size ?

> How would you choose a window size ?

> Are there large differences between insulation scores of NPC and mESC ?

## Directionality index

Directionality index measures the assymetry of contacts around a point. It was proposed by Nixon et al. 2012 and is often used to detect TADs.

## A/B Compartments

A/B compartments reflect types of chromatin (euchromatin / heterochromatin) which are isolated from each other. These compartments form characterstic "plaid" like patterns on Hi-C maps. The compartments coordinates can be extracted using PCA on detrended intrachromosomal matrices.

Since compartments are long range interactions, it is often better to use a lower resolution.

## Saddle plots

Saddle plots are a way to visually represent the intensity of the compartment signal.